<font size="6">Архитектуры CNN</font>

# Базовые компоненты свёрточных сетей



Мы рассмотрели базовые компоненты, из которых состоят современные свёрточные нейронные сети, а также техники их обучения.

<img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/basic_components_cnn.png" width = "1000">

<img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/basic_components.png" width="760">

На этом занятии рассмотрим, какие модели для классификации изображений можно построить на основе этих компонентов. 

# ImageNet Large Scale Visual Recognition Challenge

ImageNet — один из самых известных проектов в области распознавания изображений и машинного зрения. Содержит более 14 миллионов вручную размеченных изображений, принадлежащих к 1000 классам.  

Для загрузки с [официального сайта](http://image-net.org/download) необходимо запрашивать доступ, но можно загрузить данные с [Kaggle](https://www.kaggle.com/c/imagenet-object-localization-challenge/data). Однако архив занимает порядка 156Gb и не поместится на диск Colab. Поэтому воспользуемся другим репозиторием, в котором находится 1000 изображений из оригинального датасета.

P.S. Для загрузки данных, которые стали недоступны на официальных сайтах, можно использовать [Academic Torrents](https://academictorrents.com). В частности для [ImageNet](https://academictorrents.com/browse.php?search=imagenet).

In [ ]:
import torch
import random
import numpy as np

# fix random_seed
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# compute in cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from IPython.display import clear_output

# Full list of labels
#'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L09/imagenet_class_index.json

# https://github.com/ajschumacher/imagen.git
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L09/imagen.zip
!unzip imagen.zip
clear_output()

Загрузили категории:

In [ ]:
import json
import pprint

pp = pprint.PrettyPrinter(width=41, compact=True)

with open("imagenet_class_index.json") as f:
    imagenet_labels = json.load(f)

pp.pprint(
    dict(list(imagenet_labels.items())[:10])
)  # Use Pretty Print to display long dict

In [ ]:
from glob import glob
from PIL import Image
from torch.utils.data import Dataset


class MicroImageNet(Dataset):
    def __init__(self):
        super().__init__()
        # Load labels
        self.num2id = {}
        with open("imagenet_class_index.json") as f:
            imagenet_labels = json.load(f)
        w_net = {}
        # Because not world net all image codes from imagen exists in imagenet_labels
        # we need to filter this image
        for key in imagenet_labels.keys():
            wn_id = imagenet_labels[key][0]
            w_net[wn_id] = {"num": int(key), "name": imagenet_labels[key][1]}
        self.labels = []
        self.paths = []

        # Load data
        images = glob("imagen/*.jpg")
        images.sort()
        for i, path in enumerate(images):
            name = path.split("_")[2]  # Class name
            id = path.split("_")[0][7:]  # WorldNet based ID
            if w_net.get(id, None):
                self.labels.append([w_net[id]["num"], w_net[id]["name"], id])
                self.paths.append(path)

    def __getitem__(self, idx):
        im = Image.open(self.paths[idx])
        class_num = self.labels[idx][0]
        return im, class_num

    def __len__(self):
        return len(self.paths)


microImgNet = MicroImageNet()

Посмотрим на картинки:

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (15, 10)


def show(img, label_1, num, label_2=""):
    ax = plt.subplot(2, 3, num + 1)
    plt.imshow(img)
    plt.title(label_1)
    ax.set_xlabel(label_2)
    plt.axis("off")


for i in range(6):
    img, label = microImgNet[i * 6]
    name = microImgNet.labels[i * 6][1]
    show(img, name, i)

## Метрики Top 1 and Top5

**ImageNet: Large Scale Visual Recognition Challenge (ILSVRC)**

<img src ="https://edunet.kea.su/repo/src/L01_Intro/img/mp/imagenet1.png" width="700" >

На каждом изображении может быть один или несколько предметов, относящихся к одному из **1000** классов.
Для метрики *Тop5* алгоритм выдает метки 5 классов. Если предмет, относящийся к одному из этих классов, есть на изображении, то ответ засчитывается как верный.
Для *Top1* соответственно принимается только метка одного класса.

([ILSVRC](http://image-net.org/challenges/LSVRC/2017/))


<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/deep_models_modern.png"  width="900"></center>


<center><em>Лучшие результаты конкурса ILSVRC по годам.</em></center>

# Baseline (AlexNet 2012)

[ImageNet Classification with Deep Convolutional Neural Networks (Krizhevsky et al., 2012)](https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf)

Создатели: Alex Krizhevsky, Ilya Sutskever, Geoffrey E. Hinton
**University of Toronto**

Как упоминалось на первой лекции, современный бум нейросетевых технологий начался в 2012 году, когда AlexNet с большим отрывом от конкурентов победила в ImageNet.

В AlexNet есть все компоненты, которые мы рассматривали ранее. Её архитектура состоит из пяти свёрточных слоёв, между которыми располагаются pooling-слои и слои нормализации, а завершают нейросеть три полносвязных слоя.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/alexnet_architecture.png"  width="800"></center>

<center><em>Архитектура сети AlexNet</em></center>

[Подробнее про AlexNet](https://www.kaggle.com/code/blurredmachine/alexnet-architecture-a-complete-guide)

Можно заметить, что нейросеть состоит из **двух работающих параллельно нейросетей**, которые обмениваются информацией после 2-го и 5-го сверточного слоя и в полносвязных слоях. Это было необходимо, т.к. для обучения использовались **GPU GTX580** с 3 ГБ видеопамяти. Авторы архитектуры использовали две видеокарты, работающие параллельно. На вход нейронной сети подавалось трёхканальное изображение с пространственными размерами $224 \times 224$ пикселя, к которому применялись $96$ свёрток с ядром $11 \times 11 \times 3$ и сдвигом 4. Веса, соответствующие первым 48 свёрткам, хранились на первой видеокарте, а оставшиеся 48 свёрток были связаны со второй видеокартой.


- **Пространственные размеры** карты признаков резко сжимаются: $224\times224 \longrightarrow 55\times55 \longrightarrow 27\times27 \longrightarrow 13\times13 \longrightarrow 13\times13 \longrightarrow 13\times13$. 

- При этом увеличивается **количество сверток** (фильтров) в каждом слое: $96 (48\times2) \longrightarrow 256 (128\times2) \longrightarrow 384 (192\times2) \longrightarrow 256 (128\times2)$. 

- На выходе нейросети стоят два **полносвязных слоя**, формирующие ответ (в **ImageNet** 1000 классов).

**AlexNet** невозможно напрямую использовать для классификации **CIFAR-10**. Если так агрессивно уменьшать изображение размером $32\times32$ px, то в определенный момент в него просто не поместится следующий фильтр, который нужно применить. Можно сделать **resize** изображения с $32\times32$ до $224\times224$, но это не самый рациональный способ использования вычислительных ресурсов. 

**Структура** некоторых (особенно старых) сетей, заточенных под **ImageNet**, напрямую **зависит от размера изображений**. В более современных сетях есть слой [Adaptive Average Pooling](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html), позволяющий решить эту проблему, но о нем мы расскажем чуть позже. 

Такая архитектура показала прорывную точность: ошибка упала с **20%** до **15.4%**.

<center><em><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/alexnet_architecture_with_parameters.png"></em></center>
<center><em>Архитектура сети AlexNet c параметрами слоев</em></center>

Нестандартные решения, принятые разработчиками **AlexNet**:
* Использование **ReLU** в качестве функции активации. Предшественники использовали ограниченные функции, типа сигмоиды или тангенса, которые приводили к затуханию градиента.
* Использование авторской **нормализации**. Позже было показано, что их нормализация не давала значительного прироста скорости обучения и качества модели. В современных моделях чаще используется Batch-нормализация. 
* Использование различных **методов предобработки** входных данных, таких как повороты, увеличение, зашумление и т.д. для увеличения разнообразия входных данных. Это называется аугментация, и о ней вы узнаете в других лекциях. 
* Использование **Dropout** = 0.5 (при обучении отключается случайная половина нейронов).
* Использование **SGD Momentum** = 0.9. Для обеспечения сходимости авторы использовали эвристику для **понижения скорости обучения** (learning rate)
* **Скорость обучения** стартует со значения $10^{-2}$ и уменьшается в 10 раз, когда качество перестает расти.
* **L2 регуляризация** с весом $5 \cdot 10^{-4}$.
* Использование **ансамбля** из 7 обученных из различного начального состояния моделей. Результат работы моделей усредняется. Данное решение увеличило точность с 81.8% до 84.6%.


Сравним PyTorch реализацию с оригинальной. В чем отличия? 

In [ ]:
from torchvision import models

alexnet = models.alexnet(weights="AlexNet_Weights.DEFAULT")

In [ ]:
from torchsummary import summary

print("AlexNet architecture")
print(summary(alexnet, (3, 224, 224), device="cpu"))
print(alexnet)

В PyTorch версии:
- отсутствуют нормализации, 
- отсутствует разделение на две видеокарты, 
- немного изменено количество сверток на некоторых слоях,
- добавлен слой Adaptive Average Pooling.

Количество слоев и размеры сверток те же. 

Проверим, как работает: 

In [ ]:
import torch
import torchvision.transforms.functional as F


def img2tensor(img):
    t = F.to_tensor(img)
    # t = F.normalize(t, (0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    return t


def catId2names(nums):
    titles = []
    for num in nums:
        titles.append(imagenet_labels[str(num.item())][1])
        titles.reverse()
    return ", ".join(titles)


for i in range(6, 12):
    img, label = microImgNet[i * 6]
    tensor = img2tensor(img)
    out = alexnet(tensor.unsqueeze(0))  # Add batch dimension
    labels_num = torch.argsort(out[0])  # Ascending order
    weights = out[0][-5:]
    predicted = catId2names(labels_num[-5:])  # Top 5
    titles = []
    name = microImgNet.labels[i * 6][1]
    show(img, name, i - 6, predicted)

## Тюнинг гиперпараметров (ZFnet)

[Visualizing and Understanding Convolutional Networks (Zeiler et al., 2013)](https://arxiv.org/abs/1311.2901)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/zfnet_architecture.png"  width="1000"><center>
<center><em>Архитектура сети ZFNet </em><center>



<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/zfnet_architecture_2013.png"  width="1000"></center>
<center><em>Архитектура сети ZFNet </em></center>


**Тюнингованный AlexNet**

Нейросеть **ZFNet**, созданная учеными из Йорского университета, в 2013 году выиграла соревнования, достигнув результата 11.7%. В ней **AlexNet** использовалась в качестве основы, но с изменёнными параметрами и слоями.

Отличия от **AlexNet** небольшие:
* Немного поменялись пространственные размеры фильтров (было 11, стало 7);
* Увеличилось общее количество фильтров;

Количество слоев и общая структура сети, когда слои свёртки и пулинга чередуются друг с другом, а затем идут два полносвязных слоя, сохранились.

# Базовый блок (VGGNet 2014)


[Very Deep Convolutional Networks for Large-Scale Image Recognition (Simonyan et al., 2014)](https://arxiv.org/abs/1409.1556)

Karen Simonyan and Andrew Zisserman (Visual Geometry Group — **Oxford**)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/vggnet_architecture.png"  width="400" rotate="180"></center>
<center><em>Сравнение архитектур сетей AlexNet, VGG16 (использует 16 слоев) и VGG19 (использует 19 слоев)</em></center>




[Краткое описание VGGNet](https://www.robots.ox.ac.uk/~vgg/research/very_deep/)

В 2014-ом году в Оксфорде была разработана модель **VGGNet**.

В архитектуре **VGGNet** появляется идея повторения стандартных блоков и увеличения глубины нейросети повторением таких блоков (**stacking**). На момент создания данной архитектуры идея увеличения глубины нейросети для получения лучших результатов не была очевидна. Стандартный блок **VGGNet** состоит из нескольких слоев свертки (от 2 до 4) и max-pooling слоя.

Существует несколько вариантов **VGGNet** архитектуры. Самые известные **VGG11**, **VGG16** и **VGG19**. Цифра ставится по количеству слоев с обучаемыми весами: сверточных и полносвязных слоев. 

На изображении выше показаны сети **AlexNet** и две версии **VGG16** и **VGG19** с 16 и 19 обучаемыми слоями соответственно. На соревнованиях победила более глубокая **VGG19**, показавшая процент ошибок при классификации top-5 7.3% (у **AlexNet** 15.4%).

Интересные факты о **VGGNet**:
* данная архитектура получила 2-е место в задаче классификации и 1-е место в задаче локализации (классификация и обводка объекта ограничивающей рамкой) на **ImageNet**.
* авторы **VGGNet** отказались от слоев нормализации.
* архитектуры **VGG16** или **VGG19** продолжают применяться для решения ряда прикладных задач.

Особенности архитектуры **VGGNet**:
* все свёрточные слои имеют фильтры с рецептивным полем размера $3\times3$;
* сверточные слои  объединены в блоки, состоящие из некоторого количества свёрток с разным (постепенно увеличивающимся от блока к блоку) количеством фильтров;
* между блоками располагаются слои пулинга.
 
Появление "стандартных" блоков внутри модели — важное нововведение. Идея базового блока внутри сети будет достаточно широко использоваться дальше.

Разберем на коде:

In [ ]:
from torchvision import models

vgg = models.vgg16(
    weights=None
)  # Change on True if you want to use VGG to predict something
print(vgg)

Чтобы понять смысл использования сверток $3\times3$, необходимо познакомиться с понятием рецептивного поля и научиться оценивать количество необходимых вычислительных ресурсов.

### Вычислительные ресурсы



Какие ресурсы нужны для работы нейронной сети? 
* Память для хранения промежуточных представлений
* Память для обучаемых параметров
* Вычислительные ресурсы


<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/weights_memory.png"  width="650"></center>
<center><em>Пример свертки с фильтрами малого размера.</em></center>

Рассмотрим один сверточной слой (см. картинку выше).

$\color{orange}{\text{Оранжевый тензор}}$ — карта признаков, поступившая на вход сверточного слоя. Параметры $H_{in}$, $W_{in}$, $C_{in}$ — длина, ширина и глубина (число каналов изображения или фильтров в предыдущем сверточном слое) входного тензора. Данный тензор не учитывается в затрачиваемых ресурсах, т.к. информация сохранена в предыдущем слое.

**Память для хранения промежуточных представлений** определяется **количеством элементов на выходе слоя** ($\color{blue}{\text{синий тензор}}$). Они хранятся для вычисления следующих слоев и градиентов. Необходимая память рассчитывается как:

$$\large H_{out}⋅W_{out}⋅C_{out}⋅n_{byte},$$

где $H_{out}$, $W_{out}$, $C_{out}$ — длина, ширина и глубина (число фильтров в свертке) выходной карты признаков, а $n_{byte}$ — количество байт для хранения одного элемента (4 для float 32).

**Память для обучаемых параметров** определяется **весами и смещением** (bias) фильтров свертки ($\color{green}{\text{зеленые тензоры}}$). Необходимая память рассчитывается как:

$$\large (K_h⋅K_w⋅C_{in}⋅C_{out} + С_{out})⋅n_{byte},$$

где $K_h$, $K_w$, $C_{in}$ — длина, ширина и глубина одного фильтра, $C_{out}$ — число фильтров в свертке.

Для оценки необходимых **вычислительных ресурсов** посчитаем **количество операций** сложения и умножения при прямом проходе. Каждое число выходного тензора является результатом применения фильтра свертки к некоторому рецептивному полю входного тензора. Количество операций можно оценить как произведение количества элементов на выходе слоя на размер одного фильтра: 

$$\large (H_{out}⋅W_{out}⋅C_{out})⋅(K_h⋅K_w⋅C_{in}).$$


**Оценка памяти для хранения параметров слоя:**

In [ ]:
import torch.nn as nn

conv_sizes = [11, 7, 5, 3]

for conv_size in conv_sizes:
    conv_layer = nn.Conv2d(3, 64, conv_size, stride=1, padding=1)
    print("Convolution size: %ix%i" % (conv_size, conv_size))
    for tag, p in conv_layer.named_parameters():
        print("Memory reqired for %s: %.2f kb" % (tag, (np.prod(p.shape) * 4) / 1024))

**Оценка количества вычислительных операций (для одной свертки):**

**11x11:** (224 * 224 * 64) * (3 * 11 * 11) = 9 633 792 * 11 * 11 =  9 633 792 * 121  = 1 165 688 832

**7x7:** (224 * 224 * 64) * (3 * 7 * 7) = 9 633 792 * 7 * 7 = 9 633 792 * 49 = 472 055 808

121/49  = ~2.5 раза меньше чем **11x11**

**5x5:** (224 * 224 * 64) * (3 * 5 * 5) = 9 633 792 * 5 * 5 = 9 633 792 * 25 = 240 844 800

49/25 = ~2 раза меньше чем **7x7**

**3x3:** (224 * 224 * 64) * (3 * 3 * 3) = 9 633 792 * 3 * 3 = 9 633 792 * 9 = 86 704 128

25/9 = ~2.7 раза меньше чем **5x5**


### Фильтры 3x3

**Рецептивное поле** — участок входной карты признаков (входного тензора), который при прохождении одного или нескольких слоев нейронной сети формирует один признак на выходе (одно число выходного тензора). **Рецептивное поле** можно назвать “полем зрения”.

Интуитивно кажется, что чем больше “поле зрение”, тем лучше обобщающая способность свертки. Авторы **VGG** решили отказаться от свертки **5x5** и заменить ее двумя свертками **3x3**.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/receptive_field_size.png"  width="850"></center>
<center><em>Две свёртки 3×3 вместо одной 5×5.</em></center>


Рассмотрим **рецептивное поле** свертки **5x5** ($\color{green}{\text{зеленый квадрат}}$ на A1). Применение свертки **5x5** к этому квадрату даст 1 признак на выходе ($\color{red}{\text{красный квадрат}}$ на A3). Применение к этому же **рецептивному полю** свертки **3x3** ($\color{orange}{\text{оранжевые квадраты}}$ на A1) даст на **3x3** признака ($\color{red}{\text{красный квадрат}}$ на A2), применение второй свертки **3x3** позволит получить 1 признак на выходе ($\color{red}{\text{красный квадрат}}$ A3). 

**Итого:** одна свертка **5x5** имеет то же **рецептивное поле**, что две свертки **3x3**.

При этом применение двух сверток **3x3** дает ряд преимуществ:

* меньшее количество параметров. При применении свертки **5x5** необходимо обучать $5⋅5⋅C_{in}⋅C_{out} = 25⋅C_{in}⋅C_{out}$  весов, где $С_{in}$ — глубина входной карты признаков, $C_{out}$ — количество фильтров сверточного слоя. При применении двух сверток **3x3** обучается $2⋅(3⋅3⋅C_{in}⋅C_{out}) = 18⋅C_{in}⋅C_{out}$  весов.
* между свертками **3x3** добавляется дополнительный **слой активации**, который позволяет формировать более сложные признаки.

**2 маленьких фильтра работают как один большой или даже лучше!**

Аналогично 3 свертки **3x3** могут заменить одну свертку **7x7**.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/receptive_field_size_example.png"  width="850"></center>
<center><em>Три свёртки 3×3 вместо одной 7×7.</em></center>


### Память для хранения параметров (VGG16)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/memory_and_parameters_vgg16.png"  width="800"></center>
<center><em>Архитектура VGG16 с параметрами слоев </em></center>

Благодаря такой экономии получилось сделать большую по тем временам сеть (16 слоев).  Тем не менее, несмотря на применённые способы уменьшения вычислительной сложности и снижение числа параметров, сеть все равно получилась огромной (16-слойная версия сети VGG расходует в 25 раз больше дорогой памяти GPU, нежели AlexNet).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/compare_parameters_alexnet_and_vgg16.png"  width="900"></center>
<center><em>Сравнение параметров AlexNet и VGG16.</em></center>


* Основная часть памяти расходуется на большие свёртки в начальных слоях, где пространственные размеры (ширина и высота) велики;

* Больше всего весов в полносвязных слоях;

* Вычислительные ресурсы нужны в первую очередь для сверток.

VGG-16 получилась существенно больше по сравнению с и так довольно объемной AlexNet, и тем более по сравнению с современными моделями.

В значительной степени с этим связано дальнейшее направление развития моделей. В следующем году ImageNet выиграла сеть под названием GoogLeNet.


### Контроль GPU

In [ ]:
!nvidia-smi

Чтобы убедиться самостоятельно в размерах сетей, установим библиотеку для мониторинга ресурсов GPU: 

In [ ]:
!pip install -q GPUtil

In [ ]:
import GPUtil as GPU
import psutil
import os


def gpu_usage():
    GPUs = GPU.getGPUs()
    # XXX: only one GPU on Colab and isn’t guaranteed
    if len(GPUs) == 0:
        return False
    gpu = GPUs[0]
    process = psutil.Process(os.getpid())
    print(
        f"GPU RAM Free: {gpu.memoryFree:.0f}MB \
    | Used: {gpu.memoryUsed:.0f}MB \
    | Util {gpu.memoryUtil*100:3.0f}% \
    | Total {gpu.memoryTotal:.0f}MB"
    )

Посмотрим, сколько памяти потребуется **VGG19** и какого размера batch можно использовать.

In [ ]:
import torchvision

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vgg19 = torchvision.models.vgg19(weights=None, progress=True)
vgg19.requires_grad = True
vgg19.to(device)

!nvidia-smi # Common GPU info

vgg19.train()

for batch_size in [1, 8, 16, 32, 64]:
    input_random = torch.rand(batch_size, 3, 224, 224, device=device)
    out = vgg19(input_random)
    print("Batch size", batch_size)
    gpu_usage()

Очистка памяти:

In [ ]:
!nvidia-smi 

In [ ]:
torch.cuda.empty_cache()
!nvidia-smi

In [ ]:
input_random = None  # del input
out = None  # del out

!nvidia-smi

# Inception module (GoogLeNet 2014)

[Going Deeper with Convolutions (Szegedy et al., 2014)](https://arxiv.org/abs/1409.4842)

В отличие от предыдущих нейросетей, **GoolLeNet** разработана в коммерческой компании с целью практического применения, поэтому основной упор был сделан на эффективность.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/inception_module_googlenet.png"  width="400"></center>
<center><em>Inception module </em></center>

**GoogLeNet** (на самом деле архитектура называется **Inception**, а **GoogLeNet** — это имя команды в соревновании  [**ILSVRC14**](https://www.image-net.org/challenges/LSVRC/2014/), названной так в честь нейронной сети Яна Лекуна **LeNet 5**) — ещё более глубокая архитектура с 22 слоями. Она содержит менее 7 миллионов параметров — в 9 раз меньше, чем у **AlexNet**, и в 20 раз меньше, чем у **VGG19**. При этом сеть оказалась немного более точной, чем **VGG19**: ошибка снизилась с 7.3% до 6.7%.

Рассмотрим, за счёт чего удалось достичь такого огромного выигрыша в ресурсах, так как многие идеи, которые впервые были использованы для **GoogLeNet**, активно применяются до сих пор.

Составной блок **GoogLeNet** называется **Inception module**. Архитектура **GoogLeNet** состоит из множества таких блоков, следующих друг за другом.

Идея **Inception module** состоит в том, чтобы производить параллельные вычисления сверток с различным размером перцептивного поля и Max Pooling, конкатенируя (объединяя, а не складывая) полученные результаты. Это позволяет выделять признаки разного размера и сложности. 

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/naive_inception_module.png"  width="750"></center>
<center><em>Наивная реализация Inception module </em></center>

На рисунке выше представлена наивная реализация **Inception module**. Данная реализация имеет очень важный недостаток: **увеличение глубины карты признаков**. За счет конкатенации выходов **сверток** и **Max Pooling** из 256 каналов на входе мы получаем 672 канала на выходе. Количество каналов изображения увеличилось более чем в 2.6 раза. За 9 таких блоков глубина увеличится более чем в 5000 раз! 

Такое решение плохо совместимо с экономией ресурсов.
 
Стоит также заметить, что слой **Max Pooling** в данной архитектуре имеет шаг 1 и не изменяет пространственные размеры карты признаков. 

In [ ]:
import torchvision

# https://pytorch.org/vision/stable/_modules/torchvision/models/googlenet.html#googlenet
# https://hackmd.io/@bouteille/Bk-61Fo8U
googlenet = torchvision.models.googlenet(init_weights=True)
print(googlenet)

**Case Study: GoogLeNet**

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/naive_inception_module_with_dimension_reduction.png"  width="850"></center>
<center><em>Наивные реализации Inception module и Inception module с уменьшением размерности</em></center>


Как мы уже показали, при использовании наивной реализации **Inception module** количество фильтров возрастает от слоя к слою. 

Чтобы этого избежать, введены так называемые **«бутылочные горлышки»** — слои с фильтром **1×1**, уменьшающие глубину изображения. Благодаря им удалось достичь того, чтобы количество каналов на входе и на выходе либо не менялось, либо менялось только в моменты, когда это необходимо.

Интересно, что слой свертки **1x1** ставится после слоя **Max Pooling**. Это позволяет более эффективно преобразовывать признаки. 

## 1x1 Convolution





<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/1x1_convolution.png"  width="550"></center>
<center><em>Уменьшение глубины изображения с помощью 32 фильтров 1×1.</em></center>


Свёртку $1\times1$ можно сравнить с линейным слоем полносвязной нейронной сети. Мы берем вектор из карты признаков (столбик $1\times1\times64$ на картинке) и домножаем на матрицу весов (одну для всех векторов, в данном случае матрица будет $64\times32$, т.к. она составлена из $32$ фильтров с размерами $1\times1\times64$), чтобы получить вектор на выходе (столбик $1\times1\times32$ на картинке).

Это одновременно формирует более сложные признаки, собирая информацию с различных сверток и **Max Pooling**, и позволяет сократить их количество. 

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/naive_inception_module_with_dimension_reduction_parameters.png"  width="700"></center>
<center><em> Реализация Inception module с уменьшением размерности </em></center>


Количество параметров уменьшается в два с лишним раза по сравнению с наивной реализацией. Сеть получается значительно экономичнее.

Использование таких модулей и отсутствие полносвязных слоёв делают GoogLeNet очень эффективной и достаточно точной сетью. Но это далеко не все нововведения, которые появились в этой модели.

## "Stem network"

В составе **GoogLeNet** есть небольшая подсеть — **Stem Network**. Она состоит из трёх свёрточных слоёв (первый с большим фильтом) с двумя pooling-слоями и располагается в самом начале архитектуры. Цель этой подсети — быстро и сильно уменьшить пространственные размеры (сжать изображение перед параллельной обработкой), чтобы минимизировать количество элементов в слоях.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/stem_network.png" width="1024"></center>
<center><em>Архитектура GoogLeNet. В первых слоях быстро уменьшаются пространственные размеры. </em></center>

## Global Average Pooling

Отдельного внимания заслуживает завершающая часть сети. В **AlexNet** и **VGGNet** мы привыкли видеть в конце сети вытягивание карты признаков в вектор и два полносвязных слоя. В **GoogLeNet** один из полносвязных слоев заменен на 
**Global Average Pooling**. Данный слой экономичнее полносвязного. Разберемся, почему.

[Network In Network, Lin et al., 2013](https://arxiv.org/abs/1312.4400)



<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/global_average_pooling.png"  width="570"></center>
<center><em>Global Average Pooling</em></center>

[Блог-пост про Global Average Pooling](https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/)

На выходе последнего **Inception module** формируется карта признаков $7\times7\times1024$. Если вытянуть их в линейный слой, получится более 50 тысяч признаков.

Идея слоя **Global Average Pooling** (GAP) в том, что все пространственные размеры, какими бы они ни были (например, $7\times7$, как в **GoogLeNet**, или $6\times6$, как на картинке), сворачиваются в $1\times1$.

Мы берем среднее значение независимо по каждому каналу полученной карты признаков. 

Ранее считалось, что применение **Global Average Pooling** в составе архитектуры сверточной нейронной сети (CNN), то есть осуществление поканального усреднения пространственных измерений тензора, приведёт к полной потере пространственной информации о переданном сети объекте. Тем не менее, последние исследования ([Global Pooling, More than Meets the Eye: Position Information is Encoded Channel-Wise in CNNs (Islam et al., 2021)](https://arxiv.org/abs/2108.07884)) показывают, что после такого преобразования часть пространственной информации всё же сохраняется. Несмотря на то, что семантическая информация (например информация о точных границах объекта), очевидно, полностью утрачивается после Global Average Pooling, информация об абсолютном положении объекта на исходном изображении сохраняется и оказывается закодированной порядком следования компонент в оставшемся векторе усреднённых фильтров.

Применяя **GAP**, мы сократили количество признаков в 49 раз! Кроме того, **Global Average Pooling** уменьшает переобучение, т.к. мы избавляемся от влияния менее важных признаков.

Итого плюсы **GAP**:

* меньше весов;
* независимость от размера входа;
* регуляризация;
* уменьшение числа параметров;
* препятствует переобучению.

In [ ]:
import torch.nn as nn
from PIL import Image
import torch


def file2tensor(filename):
    img = Image.open(filename)
    t = torchvision.transforms.functional.to_tensor(img)
    t = torchvision.transforms.functional.normalize(
        t, (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
    )
    return t


class CNNfromHW(nn.Module):
    def __init__(self, conv_module=None):
        super().__init__()
        self.activation = nn.ReLU()
        self.conv1 = nn.Conv2d(3, 16, 5, padding=2)  # 16xHxW
        self.pool = nn.MaxPool2d(2, 2)  # 16 x H/2 x W/2
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # 32 x H/2 x W/2
        self.gap = nn.AdaptiveAvgPool2d((1, 1))  # Any spatial size -> 32x1x1
        self.fc = nn.Linear(32, 10)

    def forward(self, x):
        print("Input shape", x.shape)
        x = self.conv1(x)  # 16xHxW
        x = self.pool(x)  # 16 x H/2 x W/2
        x = self.conv2(x)  # 32 x H/2 x W/2
        x = self.activation(x)  # Any spatial size -> 32x1x1
        x = self.gap(x)
        scores = self.fc(x.flatten(1))
        print("Output shape", scores.shape)
        return scores


print("CIFAR10 like")
input_random = torch.rand(1, 3, 32, 32)
model_with_gap = CNNfromHW()
out = model_with_gap(input_random)


print("Arbitrary size")
# Different sizes work too!
aramdillo_t = file2tensor("imagen/n02454379_10511_armadillo.jpg")
out = model_with_gap(aramdillo_t.unsqueeze(0))

## Затухание градиента

**GoogLeNet: дополнительный классификатор**

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/googlenet_additional_classificator.png"  width="900"></center>
<center><em>Архитектура GoogLeNet. Отдельно вынесен блок — дополнительный классификатор.</em></center>


Помимо основного классификатора на выходе сети добавлены два дополнительных классификатора, встроенные в промежуточные слои. Они понадобились для того, чтобы улучшить обратное распространение градиента, потому что без батч-нормализации в таких глубоких сетях градиент очень быстро затухал, и обучить сеть такого размера было серьёзной проблемой.

Обучение VGG осуществлялось непростым способом: сначала обучали 7 слоев, затем добавляли туда следующие и обучали это вручную. Без использования батч-нормализации вряд ли получится повторить результат.

Google подошел более системно, добавив дополнительные выходы, которые способствовали тому, чтобы градиент меньше затухал. Благодаря этому удалось решить серьёзную на тот момент проблему, которая ограничивала возможность обучения глубоких моделей.
Статья про батч-нормализацию появилась как раз в 15-ом году, видимо, уже после выхода этой модели.

# Batchnorm (революция глубины)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/deep_models.png"  width="750"></center>
<center><em>Победители ImageNet Large Scale Visual Recognition Challenge (ILSVRC).</em></center>

В 2015 году произошла революция в глубине нейронных сетей.


[Deep Residual Learning for Image Recognition (He et al., 2015)](https://arxiv.org/abs/1512.03385)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/resnet_20_and_56_layers.png"  width="550"></center>
<center><em>График ошибки на обучающей и текстовой выборках двух модификаций  
VGGNet с 20 и 56 слоями соответственно в ходе процесса обучения. </em></center>

Проблема увеличения глубины нейронных сетей была связана не только с отсутствием батч-нормализации. На слайде выше приведено сравнение графиков обучения 56-слойной и 20-слойной сетей Microsoft, построенных на принципах VGGNet. Как видно из графиков, у 56-слойной сети и на тренировочном, и на тестовом датасете ошибка больше, чем у 20-слойной. Казалось бы, сеть, состоящая из большего количества слоёв, должна работать как минимум не хуже, чем сеть меньшего размера. К сожалению, проблема **затухания градиента** в этом случае не позволяет эффективно обучить более глубокую сеть.

В 2015 году соревнования выиграла сеть **ResNet**, архитектура которой предлагала новый подход к решению проблемы обучения глубоких сетей. Она состояла из 152 слоёв и снизила процент ошибок до 3,57%. Это сделало её почти в два раза эффективнее **GoogLeNet**.

При обучении нейронной сети методом обратного распространения ошибки модуль градиента постепенно уменьшается, проходя через каждый из слоёв сети. В глубоких сетях "длина" такого пути оказывается достаточной, чтобы модуль градиента стал мал и процесс обучения фактически остановился. В архитектуре **ResNet** такая проблема решается отказом от простого последовательного соединения слоёв (stacking) в пользу создания дополнительных связей в вычислительном графе нейронной сети, через которые градиент смог бы распространяться, минуя свёрточные слои и таким образом не затухая. 

Сеть архитектуры **ResNet** состоит из набора так называемых **Residual Block**-ов. В данном блоке тензор входных признаков пропускается через пару последовательно соединённых свёрточных слоёв, после чего полученный результат складывается поканально с этим же неизменённым входным тензором. Свёрточные слои в таком блоке аппроксимируют не саму функциональную зависимость между входным и выходным тензором, а разность (анг. residual) между такой искомой зависимостью и тождественным преобразованием.

# Skip connection (ResNet 2015)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/resudial_connection.png"  width="700"></center>
<center><em>Сравнение Residual Block с "обычными" слоями</em></center>


Идея **Residual Block** состоит в уточнении набора признаков на каждом блоке. Вместо того, чтобы перезаписывать признаки, мы добавляем к выходу предыдущего блока уточнение, сформированное на этом блоке.  

Бонусы **Residual Block**:
- канал суммирования позволяет градиенту легко распространяться без затухания;
- не теряется информация о важных признаках, выделенных на предыдущем слое.

## Архитектура ResNet

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/resnet_architecture.png"  width="730"></center>
<center><em>Архитектура ResNet. Используется стаккинг residual blocks, каждый residual block состоит из двух 3x3 conv слоев, переодически количество фильтров в свертках удваивается, а размер выходных слоев уменьшается в 2 раза (за счет stride = 2). Добавился дополнительный conv слой в начале сети. Отсутствуют FC слои (кроме последнего, который предсказывает класс).</em></center>


Из таких блоков можно построить очень глубокую сеть (были эксперименты из 1000 слоев). Для решения конкретной задачи победы на **ImageNet** хватило 150 слоев (добавление большего количества блоков уже не давало прироста точности).

В **ResNet** используются многие идеи, которые присутствовали в предыдущих моделях: 
- вначале изображение резко уменьшается,
- дальше используются блоки $3\times3$, как в **VGGNet**, 
- применяется **Average Pooling** вместо полносвязного слоя. 

Блоки состоят из конструкций, изображенных выше: 
- две свёртки $3\times3$,
- batch normalization после каждой свертки,
- функции активации ReLU,
- прибавление результата предыдущего слоя.

## ResNet: bottleneck layer

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/resnet_bottleneck_layer.png"  width="450"></center>
<center><em>В более глубоких сетях (ResNet-50+) для повышения эффективности используется слой "bottleneck" (похожим образом, что и в GoogLeNet)</em></center>

Стоит отметить, что сеть **ResNet-152** имеет существенно меньше обучаемых параметров, чем **VGG-19** (58 миллионов против 144 миллионов).

Это достигнуто за счёт того, что в более глубоких сетях, помимо упомянутых Residual Block-ов с двумя свёрточными слоями с ядром $3\times3$, применялся более эффективный блок — **bottleneck**, состоящий из:
- свёртки $1\times1$ с уменьшением количества фильтров,
- свёртки $3\times3$ с маленьким количеством фильтров,
- свёртки $1\times1$ с восстанавливаемым до начальных значений количеством фильтров, чтобы к ним можно было прибавить вход.

## ResNet реализация в PyTorch




Код для базового блока и для **bottleneck** для сетей с количеством слоев до 50.

In [ ]:
import inspect
import torchvision.models.resnet as resnet

# BasicBlock
code = inspect.getsource(resnet.BasicBlock.forward)
print(code)

In [ ]:
# Bottleneck
code = inspect.getsource(resnet.Bottleneck.forward)
print(code)

Так выглядит в коде этот базовый блок для сетей со слоями до 50 слоёв: свёртка, батч-нормализация, активация, свёртка, батч-нормализация. 

Если некоторый параметр (downsample) задан, то вызывается **downsample**: это свёртка $1\times1$ с шагом, соответствующим уменьшению размерности, и батч-нормализация.


[Модель ResNet на Torchvision](https://pytorch.org/vision/stable/_modules/torchvision/models/resnet.html#resnet18)


In [ ]:
from torchvision import models

resnet = models.resnet18(weights=None)
print(resnet)

## Обучение ResNet

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/resnet_18_and_34_layers.png"  width="550"></center>
<center><em>График изменения ошибки при обучении двух модификаций ResNet с 34 и 18 слоями соответственно</em></center>


При обучении **ResNet** шаг обучения понижали вручную, когда точность выходила на плато.

Помимо того, что **ResNet** c огромным отрывом выиграла ImageNet у моделей прошлого года, она стала первой моделью, превысившей точность человеческой разметки. Решения на базе этой архитектуры также стали победителями на соревнованиях по детектированию и сегментации.

# Grouped Convolution (ResNeXt 2016)

[2016 Aggregated Residual Transformations for Deep Neural Networks (Xie et al., 2016)](https://arxiv.org/abs/1611.05431)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/compare_resnet_and_resnext_blocks.png"  width="550"></center>
<center><em>Сравнение блоков ResNet и ResNeXt. ResNeXt создан тем же коллективом авторов, что и ResNet. Увеличение ширины residual block достигается путем использования параллельных веток "cardinality"), которые по духу схожи с Inception module. </em></center>

Следующая модель — **ResNeXt**. Эта сеть выиграла **ImageNet** в следующем году. Идея немного напоминает блок **Inception** в **GoogLeNet**: обрабатывать не сразу все каналы, а распараллелить обработку на несколько групп. 

Подобное было еще в **AlexNet**, но это делалось вынужденно, потому что **AlexNet** обучали на двух видеокартах параллельно (модель не помещалась в память одной видеокарты на тот момент).

In [ ]:
from torchvision import models

resnext = models.resnext50_32x4d(weights=None)
print(resnext)

## Grouped Convolution

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/groupped_convolution.png"  width="600"></center>
<center><em> Groupped Convolution с двумя группами сверток. Групповые свертки позволяют обрабатывать входные данные параллельно. В качестве бонуса — уменьшение числа параметров, и дополнительная регуляризация.</em></center>



[Блог-пост про различные типы сверток](https://towardsdatascience.com/a-comprehensive-introduction-to-different-types-of-convolutions-in-deep-learning-669281e58215)

На картинке выше входная карта признаков разбивается на две группы по каналам. Над каждой группой свертки вычисляются независимо, а потом конкатенируются. Это позволяет заменить $D_{out}$ фильтров $H_{in}\times H_{in}\times D_{in}$ на $D_{out}$ фильтров $H_{in}\times H_{in}\times D_{in}/2$, сократив количество обучаемых весов в 2 раза. 

## Grouped Convolution in PyTorch


Возможность вычисления груповой свертки в PyTorch заложена в привычную нам функцию свертки `nn.Conv2d`. Количество групп задается параметром `groups`, значение которого по умолчанию равно 1. 

Сравним время, которое требуется на обычную свертку с `groups = 1` и `groups = 64` при вычислениях на CPU и GPU.

In [ ]:
# CPU test
from torch import nn
import time
import torch


def time_synchronized():
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    return time.time()


input_random = torch.rand(8, 512, 112, 112)
start = time_synchronized()
normal_conv = nn.Conv2d(512, 1024, 3, groups=1)
out = normal_conv(input_random)
tm = time_synchronized() - start
print(f"Normal convolution take  {tm} sec.")

start = time_synchronized()
groupped_conv = nn.Conv2d(512, 1024, 3, groups=64)
out = groupped_conv(input_random)
tm = time_synchronized() - start
print(f"Groupped convolution take  {tm} sec.")

In [ ]:
# GPU test
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
start = time_synchronized()
normal_conv = nn.Conv2d(512, 1024, 3, groups=1).to(device)
out = normal_conv(input_random.to(device))
tm = time_synchronized() - start
print(f"Normal convolution take  {tm} sec.")

start = time_synchronized()
groupped_conv = nn.Conv2d(512, 1024, 3, groups=64).to(device)
out = groupped_conv(input_random.to(device))
tm = time_synchronized() - start
print(f"Groupped convolution take  {tm} sec.")

Применение групповой свертки со значением 64 дает выигрыш во времени в 21 раз на CPU и в 41 раз на GPU. Это связано с уменьшением количества вычислительных операций.  

Очистим память:



In [ ]:
input_random = None
out = None

## ResNeXt, Inception, Grouped convolution

<img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/resnext_inception_grouped_convolution.png"  width="850">


[Блог-пост про ResNeXt](https://towardsdatascience.com/review-resnext-1st-runner-up-of-ilsvrc-2016-image-classification-15d7f17b42ac)

Альтернативой для блока **ResNeXt** (рисунок слева) является блок **Inception-ResNeXt** (рисунок посередине), объединивший идеи **ResNeXt** блока и **Inception** блока. Применение таких блоков вместо **ResNeXt** позволяет немного улучшить результаты **ResNeXt**.

## <font color="orange">Обзор сети MobileNet (2017 г.)</font>

[MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications (Howard et al., 2017)](https://arxiv.org/abs/1704.04861)

**MobileNet** — архитектура нейронной сети, специально созданная для работы на CPU мобильных устройств. Высокая скорость работы первой версии **MobileNet** достигалась благодаря разделимой по глубине свертке (**Depthwise Separable Convolution**). 

Также для данной архитектуры предусматривается возможность варьировать размер входного изображения и ширину слоев. 

Функцией активации у **MobileNet** обычно выступает **ReLU6** (ограничение сверху у данной функции активации призвано облегчить дальнейшую 8 битную квантизацию нейронной сети). 

В 2018 году была предложена архитектура **MobileNetV2**, которая существенно превосходит первую версию благодаря добавлению к архитектуре инвертированных остаточных блоков (**Inverted Residual Block**). 

А в 2019 году была предложена уже **MobileNetV3**, которая была получена при помощи автоматического поиска архитектуры (**Network Architecture Search**) и дополнительно включала в себя модули **squeeze-and-excitation** и немонотонную функцию активации **swish** ("жесткая" версия которой, h-swish, так же призвана облегчить квантизацию).

Давайте разберемся, как все эти улучшения работают.

### Depthwise separable convolution

Обычная свертка в **CNN** одновременно работает с шириной, высотой и глубиной (каналами) карты признаков. Разделимая по глубине свёртка (**Depthwise separable convolution**) разделяет вычисления на два этапа:
- для каждого канала изображения используется свой отдельный сверточный фильтр глубины 1.  По сути это — **Grouped Convolution** с количеством групп, равным числу каналов. 
- после вычисляется свертка $1\times1$ с глубиной, равной глубине входной карты признаков. 

Применение разделимой по глубине свертки позволяет уменьшить количество обучаемых параметров и вычислительных операций при небольшой потере точности.

<img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/depthwise_and_separable_convolution.png"  width="650">

<img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/depthwise_separable_convolution.png"  width="650">

### Inverted residual block

Инвертированный остаточный блок (**Inverted residual block**) напоминает классический **residual block** из архитектуры **ResNet**, однако имеет ряд существенных отличий. Дело в том, что классический **residual block**,ъ из-за резкого уменьшения размерности пространства в комбинации с функцией активации **ReLU**, которая уничтожает всю информацию от отрицательных значений, приводит к потере большого количества информации. Поэтому вместо уменьшения количества слоев в *середине* происходит их увеличение. Увеличение вычислительной сложности компенсируется использованием разделимой по глубине свертки. А на входе и выходе из блока (где количество слоев уменьшается) отсутствует нелинейность (**ReLU**).

<img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/The-difference-between-residual-block-and-inverted-residual.png"  width="650">

### Swish

Эксперименты исследователей из Google показали, что функция активации [**Swish**](https://arxiv.org/pdf/1710.05941v1.pdf) улучшает точность глубоких моделей. Простая замена **ReLU** на **Swish** дала прирост 0.9% точности для **Mobile NASNet-A** (на ImageNet challenge).

 Посмотрим, что из себя представляет активация **Swish**:
 $$f(x) = x*\sigma(\beta x)$$





Мы видим сигмоиду и некий параметр $ \beta $, который может быть определен как константа или как обучаемый параметр. 

* Если $\beta = 0$, **Swish** становится линейной функцией $f(x) = \frac{x}{2} $

* Если $\beta\to \infty$ сигмоидная составляющая приближается к пороговой функции, поэтому **Swish** становится похожим на функцию **ReLU**. 

То есть, нелинейность функции **Swish** может контролироваться моделью, если $\beta$ задан в качестве обучаемого параметра.

Главное отличие **Swish** от **ReLU** — это ее немонотонность.

Ниже на графике представлено распределение значений, которые принимает $ \beta $ (являясь обучаемым параметром) при обучении **Mobile NASNet-A**. Видно, что значения распределены от 0 до 1.5, также виден резкий пик при значении = 1.  


<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/swish_b_parameter.png"  width="500"></center

На практике, чтобы не увеличивать количество обучаемых параметров, используют **Swish** со значением $\beta = 1$ . В PyTorch такая реализация вызывается `nn.SiLU()` (**Si**gmoid **L**inear **U**nit) 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn

silu = nn.SiLU()
array = np.arange(-5, 5, 0.01)
activated = silu(torch.Tensor(array))

plt.figure(figsize=(8, 4), dpi=100)
plt.plot(array, activated, label="$f(x)=x*\sigma(x)$")
plt.legend()
plt.grid()
plt.title("SiLU")
ax = plt.gca()
ax.spines["top"].set_color("none")
ax.spines["bottom"].set_position("zero")
ax.spines["left"].set_position("zero")
ax.spines["right"].set_color("none")
plt.ylim(bottom=-2)
plt.axis()
plt.show()

In [ ]:
from torchvision import models

mobilenet = models.mobilenet_v3_small(weights=None)
print(mobilenet)

Картинка из статьи 2018 года (не самая свежая), но она позволяет визуально сравнить модели по трем параметрам:
- размер модели (размер кружка),
- количество необходимых для обучения вычислительных операций (ось x),
- точность на задаче top-1 (ось y).

Здесь можно увидеть, что **VGGNet** — огромные по объёму модели, но по нынешним меркам они обладают средней точностью. Они требуют больших вычислительных ресурсов, поэтому сейчас их имеет смысл использовать разве что в учебных целях, а модели на базе **ResNet** (**ResNet-50**, **ResNet-152**) довольно хороши: в плане точности какого-то большого отрыва от них здесь не видно. Но, тем не менее, есть модели, которые работают чуть лучше. Рассмотрим их кратко, чтобы было понимание того, куда двигалась мысль в этой области.

# Сравнение моделей

Сравнение моделей по параметрам Точность vs Количество операций (Bianco et al., 2018)

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/compare_models_accuracy_vs_parameters.png"  width="800"></center>


<center><p><em>Source: <a href="https://arxiv.org/pdf/1810.00736.pdf">Benchmark Analysis of Representative Deep Neural Network Architectures (Bianco et al., 2018)</a></p> </em></center>

## Много skip connection (DenseNet 2016)

[Densely Connected Convolutional Networks (Huang et al., 2016)](https://arxiv.org/abs/1608.06993)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/densenet_architecture.png"  width="350"></center>
<center><em>Архитектура DenseNet. Используется несколько "обходов" и конкатенация вместо суммы. </em></center>



Еще один вариант, что можно сделать со слоями — добавить дополнительные связи в обход сверточных блоков, чтобы градиент проходил ещё лучше.

Можно также заменить сумму на конкатенацию. Чтобы не увеличить при этому глубину карт признаков, можно использовать свёртки $1\times1$. 

На этих двух принципах построен **DenseNet**. С точки зрения ресурсов он чуть более требовательный, чем базовый **ResNet**, и немного более точный. 

In [ ]:
from torchvision import models

densenet = models.densenet121(weights=None)
print(densenet)

## Ширина вместо глубины (WideResNet 2016)

[Wide Residual Networks (Zagoruyko et al., 2016)](https://arxiv.org/abs/1605.07146)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/compare_basic_and_wide_residual_blocks.png"  width="350"></center>
<center><em>Сравнение базового ResBlock и Широкого ResBlock, где используются $F \times k$ фильтры вместо $F$ фильтров в каждом слое </em></center>

Другой подход состоит в том, чтобы увеличивать не глубину, а ширину модели.
Авторы утверждают, что ширина (количество фильтров) **residuals** блока — значительно более важный фактор, чем глубина нейронной сети. 50-слойный **Wide ResNet** показывает лучшие результаты, чем оригинальный 152-х слойный **ResNet**. С точки зрения вычислительных ресурсов, использование ширины вместо глубины позволяет более эффективно параллелить вычисления на GPU (parallelizable).

# Squeeze-and-Excitation (SENet 2017)

[Squeeze-and-Excitation Networks (Hu et al., 2017)](https://arxiv.org/abs/1709.01507)

[Блог-пост про Squeeze-and-Excitation Networks](https://towardsdatascience.com/squeeze-and-excitation-networks-9ef5e71eacd7)

Основным элементом CNN являются фильтры, которые детектируют группы пикселей, обладающих локальной связностью (рецептивное поле). Сверточный фильтр объединяет:
- _пространственную информацию_ (_spatial-wise information_) &mdash; информацию о взаимном расположении пикселей друг относительно друга,
- _канальную информацию_ (_channel-wise information_)  &mdash; информацию о взаимоотношениях различных каналов внутри одного локального рецептивного поля.

Он не отделяет эти два типа информации друг от друга. За счет чередования сверточных слоев и операций субдискретизации (**pooling**) CNN способны получать представления изображений (**image representation**), которые распознают сложные иерархические паттерны.

Архитектуры сверточных нейронных сетей, рассмотренные нами до этого, концентрировались на поиске лучшего представления изображения за счет улучшения способов поиска зависимости между признаками в пространстве (**Inception module**, **Residual block** и т.д.), не затрагивая отношения между каналами.

__SENet (Squeeze-and-Excitation Networks)__ &mdash; архитектура нейронной сети, одержавшая победу в **ILSVRS-2017**. Создатели **SENet** предложили новую архитектуру блока, называемую __Squeeze-and-Excitation__ (**SE-блок**), целью которой является поиск лучшего представления изображения за счет моделирования взаимодействия между каналами. Идея состоит в том, что не все каналы одинаково важны, поэтому мы можем выборочно выделять из них более информативные и подавлять менее информативные, создав механизм взвешивания каналов (feature recalibration).  **SE-блок** состоит из следующих процессов:

1. "Сжатие" (**squeeze**) каждого канала до единственного числового значения с использованием global pooling. Эта процедура позволяет получить некое глобальное представление результата обработки исходного изображения, сделанного каждым из сверточных фильтров, (__global information embedding__).

2. "Возбуждение" (**excitation**) использует информацию, полученную на этапе "сжатия", для определения взаимодействий между каналами. Для этого используются два полносвязных слоя, первый из которых вводит "узкое место" (**bottleneck**), уменьшающее размерность в соответствии с параметром сжатия r, а второй восстанавливает размерность до исходной. В результате этой операции получается набор активаций, использующийся для взвешивания соответствующих каналов исходного изображения (__adaptive recalibration__).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/senet_architecture.png"  width="1000"></center>
<center><em>Архитектура сети SENet. Добавлен модуль для взвешивания признаков, используется GAP + 2 FC слоя. Победитель ILSVRC`17 с использованием ResNeXt-152 в качестве базовой архитектуры.</em></center>

Таким образом, **SE-блок** использует механизм, идейно напоминающий **self-attention**, для каналов, чьи отношения не ограничены локальным рецептивным полем соответствующих сверточных фильтров.

Описанный **SE-блок** может быть интегрирован в современные архитектуры сверточных нейронных сетей, например, входя в состав остаточного блока сети **ResNet** или **Inception** модуля, как изображено на рисунке.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/senet_inception_module.png"  width="1000"></center>
<center><em>Интеграция SE блока в современные архитектуры сверточных нейронных сетей </em></center>

In [ ]:
## Custom SE block


class SE_Block(nn.Module):
    "credits: https://github.com/moskomule/senet.pytorch/blob/master/senet/se_module.py#L4"

    def __init__(self, c, r=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(c, c // r, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(c // r, c, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        bs, c, _, _ = x.shape
        y = self.squeeze(x).view(bs, c)
        y = self.excitation(y).view(bs, c, 1, 1)
        return x * y.expand_as(x)

# Neural Architecture Search

[Neural Architecture Search with Reinforcement Learning (Zoph et al., 2016)](https://arxiv.org/abs/1611.01578)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/neural_architecture_search.png"  width="600"></center>
<center><em>Схема автоматического поиска архитектуры нейронных сетей </em></center>

Проектировать архитектуры нейронных сетей сложно — можно автоматизировать поиск архитектуры и пойти пить чай!

Для создания **MobileNetV3** использовалась система автоматического поиска архитектуры. Вот некоторые ее особенности: 
- Рекуррентная нейронная сеть (контроллер) выводит архитектуры сетей.
- У нас нет размеченных данных. Для того, чтобы понять, насколько хороша выведенная контроллером архитектура, создается и обучается  дочерняя сеть с этой архитектурой. При этом максимизируется итоговая точность обученной дочерней сети (это — обучение без учителя или обучение с подкреплением, подробнее об обучении с подкреплением мы поговорим в других лекциях).
- Т.к. дочерняя сеть долго обучается, есть смысл обучать несколько дочерних сетей параллельно. 
- После обучения партии дочерних сетей делаем градиентный шаг на сети контроллера (используя градиент политики).
- Со временем контроллер учится выдавать хорошие архитектуры!
- ОЧЕНЬ ДОРОГО!!! Каждый шаг градиента на контроллере требует обучения дочерних моделей!
- Оригинальная статья обучалась на 800 GPU в течение 28 дней!
- Последующие работы были сосредоточены на более эффективном поиске архитектур.

# Обзор сети EfficientNet (2019 г.)

[EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks (Tan et al., 2019)](https://arxiv.org/abs/1905.11946)

Идея **EfficientNet** состоит в том, чтобы получить возможность подбирать оптимальную глубину (число слоев), ширину (число каналов в слое) и разрешение (длину и ширину карты признаков) для конкретной задачи. Например, если мы берём входное изображение больше, чем обычно (например, $1024\times1024$ вместо привычных $25\times256$), то сети потребуется больше слоёв для увеличения рецептивного поля и больше каналов для захвата более тонких деталей на большом изображении.


Масштабирование происходит с помощью составного коэффициента (**compound coefficient**).

Например, если у нас есть возможность использовать в $2^N$ больше вычислительных ресурсов, то мы можем просто увеличить глубину сети на $\alpha^N$, ширину на $\beta^N$ и размер изображения на $\gamma^N$, где $\alpha$, $\beta$ и $\gamma$ — постоянные коэффициенты, определяемые grid search на исходной немасштабированной модели.


Базовая сеть **EfficientNet** основана на инвертированных узких остаточных блоках **MobileNet** в дополнение к блокам сжатия и возбуждения (**squeeze-and-excitation blocks**).

<img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/efficientnet.png"  width="900">

In [ ]:
!pip -q install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained("efficientnet-b0")
print(model)

# Добавим данных


Все пречисленные выше модели учились при помощи классического обучения с учителем: на размеченных данных из ImageNet.

Однако данных как в открытом так и закрытом доступе с каждым годом становиться все больше и больше. Логично предположить что из этих данных можно извлечь дополнительную информацию которая поможет лучше классифицировать старый добрый ImageNet.

Модели алгоритмы о которых пойдет речь дальше, тем или иным образом используют эти внешние данные. 


## Дистилляция

[Knowledge distillation](https://en.wikipedia.org/wiki/Knowledge_distillation)

Техника кторая используется для обучения одное модели(маленькой) при помощи другой (большой).

Представим что у нас нет больше доступа к ImageNet, но на диске осталась модель которую на нем учили,  например [Efficientnet_b7](https://pytorch.org/vision/stable/models/generated/torchvision.models.efficientnet_b7.html#torchvision.models.efficientnet_b7) Модель большая и точная, а нам нужно решить некторую простую задачу например отличать котов от собак. Но делать это на мобильном устройстве и очень быстро.


### Hard targets
Можно воспользоваться большой моделью, и с ее помощью разметить имеющиеся неразмеченные изображения  количество изображений из сети, и на этих изображениях учить нашу маленькую и быструю модель.



1.   img -> BIG_NET -> logit -> softmax -> one_hot ->label
2.   img -> small_net -> loss(logit, label) -> backprop





### Soft targets

Подход описанный выше будет работать, но можно поступить хитрее.
Предположим большая сеть классифицирует MNIST и для цифры 3 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/distilation_mnist_3.png"  width="200">


мы получили вот такой вектор логитов


In [ ]:
from matplotlib import pyplot as plt

logits = [0.1, 0.1, 0.4, 5, 0.1, 0.2, 0.1, 0.2, 3, 0.7]

plt.figure(figsize=(8, 6))
plt.bar(range(0, 10), logits)
plt.xticks(range(0, 10))
plt.show()

Самый большой логит у тройки, что верно. Но и у восьмерки восьмерки он довольно большой, так как эти цифры похожи.


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/distilation_mnist_3.png"  width="200" height="200">


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/distilation_mnist_8.png"  width="200" height="200">

То есть логиты содержат дополнительную полезную информацию.


Когда мы превратим их в вероятности, она сохраниться.

In [ ]:
import torch
from torch.nn.functional import softmax

probs = softmax(torch.tensor(logits), dim=0)

plt.figure(figsize=(8, 6))
plt.bar(range(0, 10), probs)
plt.xticks(range(0, 10))
plt.show()

А вот после того как мы превратим вектор вероятностей в метку (one_hot вектор). Эта доп. информация  пропадет при округлении.

In [ ]:
one_hot = (probs >= probs.max()).int()
print("One hot ", one_hot)

In [ ]:
from matplotlib import pyplot as plt

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4), sharey=False)


def bar(ax, y, title):
    x = range(0, 10)
    ax.bar(x, np.array(y))
    ax.set_title(title)
    ax.set_yticks([])


bar(ax1, logits, "Logits")
bar(ax2, probs, "Probs")
bar(ax3, one_hot, "Label")

plt.plot()

Что бы не терять ее можно предсказывать не метку, то есть one_hot вектор, а весь вектор вероятностей который выдает большая сеть.

## Feature extraction

<img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/feature_extraction.png"  width="650">

В процессе выполнения задачи классификации картинок мы получаем карты признаков, которые отвечают за распознавание разных особенностей и паттернов на изображениях. Эту карту признаков мы используем для решения задачи классификации. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/feature_extraction_add_x.png"  width="650">

Эти же признаки можно использовать для решения других задач. Для этого можно заменить финальные слои нейросети:

<img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/feature_extraction_backbone.png"  width="650">


На основании карт признаков можно проводить детектирование объектов, сегментацию, генерировать вектора признаков (embedding) для разных задач: сравнения изображений, распознавания лиц, трекинга и т.д.

Часть сети (слева) является достаточно универсальной.

Можно обучить веса, обучая сеть классифицировать изображения, а потом использовать предобученный фрагмент для вычисления карты признаков для любой другой задачи.

Часть нейросети, вычисляющая карту признаков, называется **backbone** (скелет, основа).

# Обучение без разметки (BYOL)

[Bootstrap Your Own Latent](https://arxiv.org/abs/2006.07733)


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/byol_performance.png"  width="400"></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/2012.12877">Bootstrap Your Own Latent</a></p> </em></center>



Методика описанная в статье использует дистилляцию для получения качественного представления данных в пространстве признаков(latent space).

**Идеи метода**: 

1.   В интернете масса неразмеченых изображений, давайте будем всячески искажать(аугментировать) их :обрезать, масштабировать, поворачивать и подавать на вход сети.

Если искажения не меняют объект на изображении до неузнаваемости, то его класс поменяться не должен, следовательно можно требовать от модели что бы вектора признаков у аугментированных изображений были похожи.

Так как меток у нас нет, то в качестве loss можно использовать расстояние между векторами признаков. У для векторов полученных з одного изображения оно должно быть меньше чем между векторами различных изображений.  Эта идея [Contrastive Loss](https://paperswithcode.com/method/supervised-contrastive-loss) которая подробно будет обсуждаться в 11й лекции.


При наивной реализации такой подход работает не слишком хорошо, так как общее количество сравнений для датасета с N изображениями и M аугментациями это $ (N*M)^2$  Поэтому приходится выбирать из всех возможных пар только часть самых сложных ([Hard example Mining](https://paperswithcode.com/method/ohem) )

2. Авторы статьи предложили решение основанное на идее дистилляции. 

Создаются две одинаковые сети (online и target).  И два различных набора аугментаций t и t`. В первую сеть подаются изображения аугментированные при помощи t во вторую при помощи t'

Предсказания моделей сравнивуются лосс подсчитыватся как рассояние между предсказаниями, но обновление весов методом градиентного спуска происходит **только у одной модели** (online) .

Веса второй(target) постепенно обновляются как [экспоненциальное скользящее среднее](https://en.wikipedia.org/wiki/Exponential_smoothing) от весов первой модели.

$\xi \leftarrow \tau \xi + (1-\tau\theta)$

где $\tau$ - скорость обновления, $\theta$ веса другой(online) модели.









<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/byol_scheme.png"  width="900"></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/2012.12877">Bootstrap Your Own Latent</a></p> </em></center>


Затем достаточно получить embedding - и для изображений из ImageNet и классифицировать их при помощи линейного классификатора.


Код https://github.com/open-mmlab/mmselfsup

# CLIP

Contrastive Language-Image Pre-training

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/clip_idea.png"  width="700"></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/2103.00020">Learning Transferable Visual Models From Natural Language Supervision</a></p> </em></center>

Для обучения модели был создан новый датасет включающий 400M пар изображение - текст.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/clip_classifier.png"  width="700"></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/2103.00020">Learning Transferable Visual Models From Natural Language Supervision</a></p> </em></center>

Для использования в качестве классификатора достаточно создать текстовый эмбеддинг для слова описывающего каждый класс, а затем сравнить его с эмбеддингом изображения.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/clip_performance.png"  width="700"></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/2103.00020">Learning Transferable Visual Models From Natural Language Supervision</a></p> </em></center>

Точность классификации не дотягивает до SOTA upervised моделей, но зато работает вообще без дообучения на различных датасетах.

# Self Attention (ViT 2020)

[Visual Transformers: Token-based Image Representation and Processing for Computer Vision (Wu et al., 2020)](https://arxiv.org/abs/2006.03677)

[Реализация](https://github.com/lucidrains/vit-pytorch)

[Блог-пост разбор  ViT](https://towardsdatascience.com/implementing-visualttransformer-in-pytorch-184f9f16f632)


**Vision Transformer** — это модель для классификации изображений, которая использует архитектуру трансформера. Попробуем разобраться, как она работает. 

В 2020 году стали появляться работы, где модели на базе архитектур трансформер смогли показать результаты лучше, чем у **CNN** моделей.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/cited_vit_accuracy.png"  width="650"></center>


<center><p><em>Source: <a href="https://arxiv.org/abs/2010.11929">An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale (Dosovitskiy et al., 2020</a></p> </em></center>

BiT — это baseline модель на базе **ResNet**, ViT — **Visual Transformer**



### Недостатки сверточного слоя

Авторы практически полностью отказались от использования сверток,  заменив их слоями **self-attention**.  Попробуем понять, почему это сработало.

Добавляя в модель свёрточный слой, мы руководствуемся резонным предположением: чем ближе пиксели на изображении, тем больше будет их взаимное влияние.

В большинстве случаев это работает:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/cnn_ok.png"  width="700"></center>

 - На слое n (красный) активируются нейроны, которые реагируют на морду и на хвост кота.

 - В карте активаций их выходы оказываются рядом, и в слое n + 1 (синий) они попадают в одну свертку, которая активируется на объектах типа "кот".

Так случается часто, но не всегда:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/cnn_fail.jpg"  width="700"></center>

На этом изображении активации нейронов, реагирующих на морду и хвост, не попадут в одну свертку на следующем слое. Это может привести к тому, что нейрон, обучившийся реагировать на кошек, не активируется.

Причиной этого является допущение ([Inductive bias](https://en.wikipedia.org/wiki/Inductive_bias)) о взаимном влиянии соседних пикселей. 

Другая проблемма сверток состоит в том что они слабо учитывают контекст.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/conv_vs_self_attention1.png"  width="700"></center>

Для нас очевидно что на кусте клубники скорее всего окажется еще одна ягода клубники а не малина, вишня или грузовик. 

### Self-attention

**Self-attention** слой лишен этого недостатка. Он обучается оценивать взаимное влияние входов друг на друга. Но как применить его к изображениям?

В статье [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale (Dosovitskiy et al., 2020)](https://arxiv.org/pdf/2010.11929.pdf) предлагается разбивать картинки на кусочки (patches) размером 16x16 пикселей и подавать их на вход модели. 

Проделаем это:

In [ ]:
URL = "https://edunet.kea.su/repo/EduNet-web_dependencies/L09/cat.jpeg"
!wget -q $URL -O image.jpg

Преобразуем изображение в тензор, порежем на фрагменты и отобразим их, используя image_grid

In [ ]:
from torchvision import utils, transforms
import matplotlib.pyplot as plt
import torch
from PIL import Image

img = Image.open("image.jpg")

transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

img = transform(img)
patches = []
sz = 64
for r in range(0, img.shape[1], sz):
    for c in range(0, img.shape[2], sz):
        patches.append(img[:, r : r + sz, c : c + sz])

patches = torch.stack(patches).type(torch.float)

img_grid = utils.make_grid(patches, pad_value=10, normalize=True, nrow=4)
plt.imshow(transforms.ToPILImage()(img_grid).convert("RGB"))
plt.axis("off")
plt.show()

На вход модели они поступят в виде вектора:

In [ ]:
plt.figure(figsize=(18, 6))
img_grid = utils.make_grid(patches, pad_value=10, normalize=True, nrow=256 // 16)
plt.imshow(transforms.ToPILImage()(img_grid).convert("RGB"))
plt.axis("off");

Затем последовательность из фрагментов изображения передается в модель, где после ряда преобразований попадает на вход слоя **self-attention**:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/self_attention.png"  width="900"></center>

Картинки приведены исключительно для наглядности, в действительности слой работает с векторами признаков, которые не визуализируются столь очевидно. Однако кэффициенты с которыми складываются вектора-признаков отражают важность каждого с учетом всех остальных входов.



Попробуем провести аналогию со свертками. Если рассмотреть сверточную сеть без слоев пулинга и других способов изменения пространственных размеров карт признаков, то можно считать такую сеть механизмом постепенно улучшаюшем качество признаков.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/conv_vs_self_attention4.png"  width="700"></center>

Каждый признак умножается на свой вес, и затем они суммируются. Важно что вклад взвешенных признаков в сумму не зависит от контекста. То есть ягода клубники лежащая на столе, где рядом с ней может быть все что угодно даст такой же вклад в сумму как и я ягода с клубничного куста.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/conv_vs_self_attention2.png"  width="900"></center>

Слой self-attention выполняет ту же задачу что и свертка: получает на вход вектор признаков и возвращает другой, более информативный.  Но делает это более умно:





<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/conv_vs_self_attention3.png"  width="900"></center>

*Вместо чисел здесь вектора, но принципильно это ничего не меняет, можно применить self-attention и к отдельным признакам (яркостям, пикселям) просто для это потребуется очень много ресурсов.*

Во первых **каждый признак** участвует в сумме, а не только те что попали в рецептивное поле фильтра.
А во вторых суммируются они с коэффициентами $a$ которые **зависят от входов**.

Для получения этих коэффициентов и нужна большая часть слоя self-attention. На рисунке выделено расным.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/conv_vs_self_attention5.png"  width="900"></center>

#### <font color="orange">Соображения относительно размера patch</font>

Трансформеры работают с последовательностями за счёт механизма внимания (**self-attention**). И чтобы подать на вход изображение требуется превратить его в последовательность.

Сделать это можно разными способами, например, составить последовательность из всех пикселей изображения. Её длина $n =  H*W$ (высота на ширину)


[Сложность вычисления](https://www.researchgate.net/figure/Compare-the-computational-complexity-for-self-attention-where-n-is-the-length-of-input_tbl7_347999026) одноголового слоя **self-attention** $O(n^2 d )$  где $n$ — число токенов и $d$ — размерность входа (embedding)  (для любознательных расчеты [тут](https://stackoverflow.com/questions/65703260/computational-complexity-of-self-attention-in-the-transformer-model)).

То есть для квадратных изображений $(H==W)$ получим $O(H^3 d )$

1. Такой подход будет очень вычислительно сложен.

2. Интуитивно понятно, что кодировать каждый пиксел относительно большим embedding-ом не очень осмысленно.


*Для тех, кто забыл, напомним что $O()$ — это Big O notation, которая отражает ресурсы, требуемые для вычисления. Так для $O(1)$ — время вычисления будет постоянным, вне зависимости от количества данных, а для $O(N)$ — расти пропорционально количеству данных.*


Разберём на примере: Допустим, мы используем трансформер для предложения длиной в 4 слова — "Мама мылом мыла раму" => у нас есть `4 токена`. Закодируем их в *embeddings* с размерностью `256`. Потребуется порядка $4^2*256 = 4096$ операций.

А теперь попробуем провернуть то же самое для картинки размерами 256 на 256.
Количество токенов 

 $256^3*256  = 256^4 =  4 294 967 296 $. Упс... Кажется, нам так никаких ресурсов не хватит — трансформеры с картинками использовать.



Посчитаем сложность для картинки размером 256x256, разбитой на кусочки по 16px. при том же размере токена (256) $n = 16$.
$16^2*256 = 256^2 = 65536 $. И впрямь! ~65000 раз меньше ресурсов требуется.

[Как устроен  self-attention](https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a)

[Self-attention слой в PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)

### Position embedding

Не теряем ли мы важной информации, разбивая изображение на фрагменты? На первый взгляд кажется, что модель сможет научиться восстанавливать порядок, в котором фрагменты шли в исходном изображении.

Всегда ли? 

Рассмотрим пример изображения, где нет ярко выраженной текстуры:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/positional_transformer_explanation.png"  width="500"></center>

На рисунке а) наковальня падает на ребенка, на рисунке б) ребенок прыгает на наковальне.
 
Суть принципиально отличается, но что будет, если составить из фрагментов любого изображения набор патчей:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/positional_vec_transformer_explanation.png"  width="500"></center>

Восстановить по нему можно будет любой из вариантов!

Так как **self-attention** блок никак не кодирует позицию элемента на входе, то важная информация потеряется.

Чтобы избежать таких потерь, информацию, кодирующую позицию фрагмента (patch),  добавляют к входным данным **self-attention** слоя в явном виде.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/linear_projection_of_flattened_patches.png"  width="600"></center>

[Методы для кодирования позиции](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/)

## Архитектура ViT

Теперь мы можем грузить наши изображения в **Vi**sual **T**ransformer.

**Self-attention** блок мы разобрали, остальные блоки модели нам знакомы:

> **MLP** (Multi layer perceptron) — Блок из одного или нескольких линейных слоев

> **Norm** — Layer Normalization

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L09/out/visual_transformer_architecture.png"  width="1000"></center>
<center><em>Архитектура Visual Transformer </em></center>



1.   Изображение режется на фрагменты (patch)
2.   Фрагменты (patch) подвергаются линейной проекции с помощью **MLP**
3.   С полученными на выходе **MLP** векторами конкатенируются **positional embeddings** (кодирующие информацию о позиции path, как и в обычном трансформере для текста)
4. К полученным векторам добавляют еще один **0***, который называют **class embedding**

Любопытно, что для предсказания класса используется только выход. Он соответствует дополнительному **class embedding**.  Остальные выходы (а для каждего токена в трансформере есть свой выход) отбрасываются за ненадобностью.

В финале этот специальный токен **0*** прогоняют через **MLP** и предсказывают классы.

## Предсказание с помощью ViT


Используем пакет [ViT PyTorch](https://pypi.org/project/pytorch-pretrained-vit/)



In [ ]:
!pip install -q pytorch_pretrained_vit

В пакете доступны несколько [предобученных моделей](https://github.com/lukemelas/PyTorch-Pretrained-ViT#loading-pretrained-models):

B_16, B_32, B_16_imagenet1k, ... 



In [ ]:
from pytorch_pretrained_vit import ViT

model = ViT("B_16_imagenet1k", pretrained=True)
model.eval()

# Load image
# NOTE: Assumes an image `image.jpg` exists in the current directory
img = transforms.Compose(
    [
        transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(0.5, 0.5),
    ]
)(Image.open("image.jpg")).unsqueeze(0)
print(img.shape)  # torch.Size([1, 3, 384, 384])

# Classify
with torch.no_grad():
    outputs = model(img)
print(outputs.shape)  # (1, 1000)

Давайте посмотрим, что нам предсказывает ViT. Для этого подгрузим dict с переводом индексов в человеческие названия:

In [ ]:
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L09/imagenet_class_index.json

In [ ]:
import json

with open("imagenet_class_index.json") as f:
    imagenet_labels = json.load(f)

И, собственно, переведем индекс в название:

In [ ]:
print(
    "Top 3 predictions:",
    [imagenet_labels[str(x.item())][1] for x in outputs.topk(3).indices[0]],
)

Ну что ж, почти (капибар в классах ImageNet 1k, как вы могли догадаться, просто нет).

## Обучение ViT

### Объем данных и ресурсов

Как следует из текста [статьи](https://arxiv.org/abs/2010.11929), **ViT**, обученный на **ImageNet**, уступал baseline CNN-модели
на базе сверточной сети (**ResNet**). И только при увеличении датасетов больше, чем **ImageNet**, преимущество стало заметным.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/cited_vit_accuracy.png"  width="400"></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/2010.11929">An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale (Dosovitskiy et al., 2020</a></p> </em></center>
 

Вряд ли в вашем распоряжении окажется датасет, сравнимый с [JFT-300M](https://paperswithcode.com/dataset/jft-300m) (300 миллионов изображений),
и GPU/TPU ресурсы, необходимые для обучения с нуля (*it could be trained using a standard cloud TPUv3 with 8 cores in approximately 30 days*)

Поэтому для работы с пользовательскими данными используется техника дообучения ранее обученной модели на пользовательских данных (**fine-tuning**).

### DeiT: Data-efficient Image Transformers

Для практических задач рекомендуем использовать эту реализацию. Авторы предлагают подход, благодаря которому становится возможным обучить модель на стандартном **ImageNet** (ImageNet1k) на одной рабочей станции за 3 дня.

*We train them on a single computer in less than 3 days. Our reference vision transformer (86M parameters) achieves top-1 accuracy of 83.1% (single-crop evaluation) on ImageNet with no external data.*

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/cited_deit_vit.png"  width="700"></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/2012.12877">Training data-efficient image transformers & distillation through attention</a></p> </em></center>



Разбор этого материала уже не входит в наш курс и рекомендуется к самостоятельному изучению.

Дополнительно:
[Distilling Transformers: (DeiT) Data-efficient Image Transformers](https://towardsdatascience.com/distilling-transformers-deit-data-efficient-image-transformers-61f6cd276a03)

Статьи, предшествовавшие появлению **ViT**:

[Non-local Neural Networks](https://arxiv.org/abs/1711.07971)

[CCNet: Criss-Cross Attention for Semantic Segmentation](https://arxiv.org/abs/1811.11721)






### <font color="orange">Использование ViT с собственным датасетом</font>

Для использования **ViT** с собственными данными рекомендуем не обучать собственную модель с нуля, а использовать уже предобученную.

Рассмотрим этот процесс на примере. Есть предобученный на **ImageNet** **Visual Transformer**, например: [deit_tiny_patch16_224](https://github.com/facebookresearch/deit)

И мы хотим использовать ее со своим датасетом, который может сильно отличаться от **ImageNet**.

Для примера возьмем **CIFAR-10**.



Загрузим модель. Как указано на [github](https://github.com/facebookresearch/deit), модель зависит от библиотеки [timm](https://fastai.github.io/timmdocs/), которую нужно установить.

In [ ]:
!pip install -q timm

Теперь загружаем модель с [pytorch-hub](https://pytorch.org/hub/):

In [ ]:
model = torch.hub.load(
    "facebookresearch/deit:main", "deit_tiny_patch16_224", pretrained=True
)

Убедимся, что модель запускается. 
Загрузим изображение:

In [ ]:
!wget -qnc https://edunet.kea.su/repo/EduNet-web_dependencies/L09/capybara.jpg -O image.jpg

И подадим его на вход трансформеру:

In [ ]:
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import torchvision.transforms as T

pil = Image.open("image.jpg")

# create the data transform that DeiT expects
imagenet_transform = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
    ]
)

out = model(imagenet_transform(pil).unsqueeze(0))
print(out.shape)
pil.resize((224, 224))

Чтобы использовать модель с **CIFAR-10**, нужно поменять количество выходов слоя, отвечающих за классификацию. Так как в **CIFAR-10** десять классов, а в **ImageNet** тысяча.

Чтобы понять, как получить доступ к последнему слою, выведем структуру модели:


In [ ]:
print(model)

Видим, что последний слой называется head и, судя по количеству параметров на выходе (1000), которое совпадает с количеством классов **ImageNet**, именно он отвечает за классификацию. 

In [ ]:
print(model.head)

Заменим его слоем с 10-ю выходами по количеству классов в CIFAR-10.

In [ ]:
model.head = torch.nn.Linear(192, 10, bias=True)

Убедимся, что модель не сломалась.

In [ ]:
out = model(imagenet_transform(pil).unsqueeze(0))
print(out.shape)

Теперь загрузим **CIFAR-10** и проверим, как дообучится модель

In [ ]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

cifar10 = CIFAR10(root="./", train=True, download=True, transform=imagenet_transform)

# We use only part of CIFAR10 to reduce training time
trainset, _ = torch.utils.data.random_split(cifar10, [10000, 40000])
train_loader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = CIFAR10(root="./", train=False, download=True, transform=imagenet_transform)
test_loader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

 Проведем стандартный цикл обучения.

In [ ]:
from torch import nn
from tqdm.notebook import tqdm_notebook

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train(model, train_loader, optimizer, num_epochs=1):
    model.to(device)
    model.train()
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for batch in tqdm_notebook(train_loader):
            inputs, labels = batch
            optimizer.zero_grad()
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

Дообучаем (**fine tune**) только последний слой модели, который мы изменили.

In [ ]:
import torch.optim as optim

model.to(device)
optimizer = optim.SGD(model.head.parameters(), lr=0.001, momentum=0.9)
train(model, train_loader, optimizer)

Проверим точность, на всей тестовой подвыборке **CIFAR-10**.

In [ ]:
@torch.inference_mode()
def accuracy(model, testloader):
    correct = 0
    total = 0
    for batch in testloader:
        images, labels = batch
        outputs = model(images.to(device))
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
    return correct / total

In [ ]:
print(f"Accuracy of fine-tuned network : {accuracy(model, test_loader):.2f} ")

Дообучив последний слой на одной эпохе с использованием 20% данных, мы получили точность ~0.75

Если дообучить все слои на 2-х эпохах, можно получить точность порядка 0.95. 

Это результат намного лучше чем тот, что мы получали на семинарах.

Для этого потребуется порядка 10 мин (на GPU). Сейчас мы этого делать не будем.
 

И одной из причин того, что обучение идет относительно медленно, является увеличение изображений размером 32x32 до 224x224. 

Если бы мы использовали изображения **CIFAR-10** в их родном размере, мы бы не потеряли никакой информации, но могли бы в разы ускорить обучение.


### Изменение размеров входа ViT

На первый взгляд, ничего не мешает это сделать: **self-attention** слой работает с произвольным количеством входов.

Давайте посмотрим, что будет, если подать на вход модели изображение, отличное по размерам от 224x224.

Для этого перезагрузим модель:

In [ ]:
def get_model():
    model = torch.hub.load(
        "facebookresearch/deit:main", "deit_tiny_patch16_224", pretrained=True
    )
    model.head = torch.nn.Linear(192, 10, bias=True)
    return model


model = get_model()

И уберем из трансформаций Resize:

In [ ]:
cifar_transform = T.Compose(
    [
        # T.Resize((224, 224)),    don't remove this line
        T.ToTensor(),
        T.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
    ]
)

# Change transformation in base dataset
cifar10.transform = cifar_transform
first_img = trainset[0][0]

model.to(torch.device("cpu"))
try:
    out = model(first_img.unsqueeze(0))
except Exception as e:
    print("Exception:", e)

Получаем ошибку.

Ошибка возникает в объекте [PatchEmbed](https://huggingface.co/spaces/Andy1621/uniformer_image_demo/blob/main/uniformer.py#L169), который превращает изображение в набор эмбеддингов.

У объекта есть свойство `img_size`, попробуем просто поменять его:

In [ ]:
model.patch_embed.img_size = (32, 32)
try:
    out = model(first_img.unsqueeze(0))
except Exception as e:
    print("Exception:", e)

Получаем новую ошибку.

И возникает она в строке  
`x = self.pos_drop(x + self.pos_embed)`

x — это наши новые эмбеддинги для CIFAR-10 картинок

Откуда взялось число 5?

4 — это закодированные фрагменты (patch) для картинки 32х32 их всего 4 (16x16) + один embedding для предсказываемого класса(class embedding).

А 197 это positional encoding — эмбеддинги, кодирующие позицию элемента. Они остались от **ImageNet**.

Так как в ImageNet картинки размера 224x224, то в каждой помещалось 14x14 = 196 фрагментов и еще embedding для класса, итого 197 позиций.



Эмбеддинги для позиций доступны через свойство:

In [ ]:
model.pos_embed.data.shape

Теперь нам надо изменить количество pos embeddings так, чтобы оно было равно 5  (количество patch + 1). 
Возьмем 5 первых:

In [ ]:
model.pos_embed.data = model.pos_embed.data[:, :5, :]
out = model(first_img.unsqueeze(0))
print(out.shape)

Заработало!

Теперь обучим модель. Так как изображения стали намного меньше, то мы можем увеличить размер batch и использовать весь датасет. Так же будем обучать все слои, а не только последний.

In [ ]:
cifar10.transform = cifar_transform
train_loader = DataLoader(cifar10, batch_size=512, shuffle=True, num_workers=2)

# Now we train all parameters because model altered
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
train(model, train_loader, optimizer)

Сильно быстрее. 
Посмотрим на результат:

In [ ]:
testset.transform = cifar_transform
print(f"Accuracy of altered network : {accuracy(model,test_loader):.2f} ")

Сильно хуже. 

Это можно объяснить тем, что  маленькие patch  ImageNet(1/196) семантически сильно отличаются от четвертинок картинок из CIFAR-10 (1/4).

Но есть и другая причина: мы взяли лишь первые 4 pos_embedding а остальные отбросили. В итоге модель вынуждена практически заново обучаться работать с малым pos_embedding, и двух эпох для этого мало. 

Зато теперь мы можем использовать модель с изображениями любого размера.

# ConvNext (2022)





<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/convnext_v1_performance.png"  width="500"></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/2201.03545">A ConvNet for the 2020s</a></p> </em></center>

###Ablation study

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/convnext_v1_ablation.png"  width="500"></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/2201.03545">A ConvNet for the 2020s</a></p> </em></center>


[ConvNeXt V2: Co-designing and Scaling ConvNets with Masked Autoencoders](https://arxiv.org/abs/2301.00808)



#Model soups (2022)

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L09/model_soups_performance.png"  width="900"></center>

<center><p><em>Source: <a href="https://arxiv.org/abs/2203.05482">Model soups: averaging weights of multiple fine-tuned models improves accuracy without increasing inference time</a></p> </em></center>

#Практические рекомендации

*  Используйте EfficientNet  качестве baseline
*  HugginFace


<font size="6">Список использованной литературы</font>

<font size="5">AlexNet</font>

[ImageNet Classification with Deep Convolutional Neural Networks (Krizhevsky et al., 2012)](https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf)

[Подробнее про AlexNet](https://www.kaggle.com/code/blurredmachine/alexnet-architecture-a-complete-guide)

<font size="5">ZFNet</font>

[Visualizing and Understanding Convolutional Networks (Zeiler et al., 2013)](https://arxiv.org/abs/1311.2901)

<font size="5">VGG</font>

[Very Deep Convolutional Networks for Large-Scale Image Recognition (Simonyan et al., 2014)](https://arxiv.org/abs/1409.1556)

[Краткое описание VGGNet](https://www.robots.ox.ac.uk/~vgg/research/very_deep/)

<font size="5">GoogLeNet</font>

[Going Deeper with Convolutions (Szegedy et al., 2014)](https://arxiv.org/abs/1409.4842)

<font size="5">Global Average Pooling</font>

[Network In Network, Lin et al., 2013](https://arxiv.org/abs/1312.4400)

[Блог-пост про Global Average Pooling](https://alexisbcook.github.io/2017/global-average-pooling-layers-for-object-localization/)

<font size="5">ResNet</font>

[Deep Residual Learning for Image Recognition (He et al., 2015)](https://arxiv.org/abs/1512.03385)

<font size="5">Fixup Initialization</font>

[Fixup Initialization: Residual Learning Without Normalization (Zhang et. al, 2019)](https://arxiv.org/abs/1901.09321)

[Блог-пост про Fixup Initialization](https://towardsdatascience.com/understanding-fixup-initialization-6bf08d41b427)

<font size="5">ResNeXt</font>

[Aggregated Residual Transformations for Deep Neural Networks (Xie et al., 2016)](https://arxiv.org/abs/1611.05431)

[Блог-пост про ResNeXt](https://towardsdatascience.com/review-resnext-1st-runner-up-of-ilsvrc-2016-image-classification-15d7f17b42ac)

<font size="5">DenseNet</font>

[Densely Connected Convolutional Networks (Huang et al., 2016)](https://arxiv.org/abs/1608.06993)

<font size="5">WideResNet</font>

[Wide Residual Networks (Zagoruyko et al., 2016)](https://arxiv.org/abs/1605.07146)


<font size="5">SENet</font>

[Squeeze-and-Excitation Networks (Hu et al., 2017)](https://arxiv.org/abs/1709.01507)

[Блог-пост про Squeeze-and-Excitation Networks](https://towardsdatascience.com/squeeze-and-excitation-networks-9ef5e71eacd7)


<font size="5">MobileNet</font>

[MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications (Howard et al., 2017)](https://arxiv.org/abs/1704.04861)

<font size="5">EfficientNet</font>

[EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks (Tan et al., 2019)](https://arxiv.org/abs/1905.11946)

<font size="5">Visual Transformer</font>

[Visual Transformers: Token-based Image Representation and Processing for Computer Vision (Wu et al., 2020)](https://arxiv.org/abs/2006.03677)

[Блог-пост разбор  ViT](https://towardsdatascience.com/implementing-visualttransformer-in-pytorch-184f9f16f632)

[An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale (Dosovitskiy et al., 2020)](https://arxiv.org/abs/2010.11929)

<font size="5">MLP-Mixer</font>

[MLP-Mixer: An all-MLP Architecture for Vision (Tolstikhin et al., 2021)](https://arxiv.org/abs/2105.01601)

[Разбор во влоге Yannic Kilcher](https://www.youtube.com/watch?v=7K4Z8RqjWIk&feature=youtu.be&ab_channel=YannicKilcher)

[Блог-пост про MLP-Mixer](https://wandb.ai/wandb_fc/pytorch-image-models/reports/Is-MLP-Mixer-a-CNN-in-disguise---Vmlldzo4NDE1MTU)

<font size="5">Разное</font>

[Блог-пост про различные типы сверток](https://towardsdatascience.com/a-comprehensive-introduction-to-different-types-of-convolutions-in-deep-learning-669281e58215)

[Benchmark Analysis of Representative
Deep Neural Network Architectures (Bianco et al., 2018)](https://arxiv.org/abs/1810.00736)

[Neural Architecture Search with Reinforcement Learning (Zoph et al., 2016)](https://arxiv.org/abs/1611.01578)
